In [1]:
from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder, labels_to_list_of_regions
from nnunetv2.imageio.nibabel_reader_writer import NibabelIOWithReorient
import json

folder_ref = '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_raw/Dataset300_mosaic_cardiovascular/labelsTr'
model_folder = '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset300_mosaic_cardiovascular/nnUNetTrainerNoMirroring__Mosaic_nnUNetPlannerResEncL_stdres_NoRsmp_24G__3d_fullres'
folder_pred = f'{model_folder}/fold_0/validation'
output_file = f'{folder_pred}/summary.json'
dataset_json = f'{model_folder}/dataset.json'
image_reader_writer = NibabelIOWithReorient()
file_ending = '.nii.gz'
ignore_label = None
num_processes = 12

with open(dataset_json) as f:
    data = json.loads(f.read())

# regions = labels_to_list_of_regions(list(data['labels'].values())[1:])

regions = list(data['labels'].values())[1:]

compute_metrics_on_folder(folder_ref, folder_pred, output_file, image_reader_writer, file_ending, regions, ignore_label,
                            num_processes)

{'metric_per_case': [{'reference_file': '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_raw/Dataset300_mosaic_cardiovascular/labelsTr/adrenal_Ki67_Seg_001_native.nii.gz',
   'prediction_file': '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset300_mosaic_cardiovascular/nnUNetTrainerNoMirroring__Mosaic_nnUNetPlannerResEncL_stdres_NoRsmp_24G__3d_fullres/fold_0/validation/adrenal_Ki67_Seg_001_native.nii.gz',
   'metrics': {1: {'Dice': 0.8295648370872513,
     'IoU': 0.7087661609745161,
     'FP': 17071,
     'TP': 56904,
     'FN': 6311,
     'TN': 41434914,
     'n_pred': 73975,
     'n_ref': 63215},
    2: {'Dice': 0.9180217561960301,
     'IoU': 0.8484660033167496,
     'FP': 978,
     'TP': 8186,
     'FN': 484,
     'TN': 41505552,
     'n_pred': 9164,
     'n_ref': 8670},
    3: {'Dice': 0.9290598585475246,
     'IoU': 0.8675180083244205,
     'FP': 5598,
     'TP': 67322,
     'FN': 4683,
     'TN': 41437597,
     'n_pred': 72920,
     'n_ref': 72005},
    4: {'